In [7]:
from openai import OpenAI
from dotenv import load_dotenv
import os
from transformers import pipeline
import torch
from langdetect import detect
import pandas as pd
load_dotenv()

True

In [2]:
huggingface_api = os.getenv("HUGGING_FACE_KEY")
openai_api=os.getenv('OPENAI_API_KEY')

In [3]:
def generate_email(api_key,prompt,tone,model='gpt-3.5-turbo'):
    if not api_key:
        return {"error": "Please enter your OpenAI API key.", "response": None}
    try:
        llm=OpenAI(api_key=api_key)

        # email response
        messages=[
            {"role": "system", "content": f"You are a professional email assistant. Generate a {tone} tone response."},
            {"role": "user", "content": prompt}
        ]

        response=llm.chat.completions.create(model=model,messages=messages,temperature=.8)
        email_response=response.choices[0].message.content

        # Subject line
        subj_msgs = [
            {"role": "system", "content": "Generate a concise subject line for this email."},
            {"role": "user", "content": email_response}
        ]
        subject=llm.chat.completions.create(model=model, messages=subj_msgs, temperature=0.7)
        subject_response=subject.choices[0].message.content

        # thread summrary
        sum_msgs = [
            {"role": "system", "content": "Provide a concise summary of the email thread."},
            {"role": "user", "content": f"Thread:\n{prompt}\n\nResponse:\n{email_response}"}
        ]
        summary=llm.chat.completions.create(model=model, messages=sum_msgs, temperature=0.7)
        thread_summary = summary.choices[0].message.content

        return {"error": None, "response": email_response, "subject": subj_msgs, "summary": thread_summary}

    except Exception as e:
        return {
            "error": str(e),
            "response": None
        }
    

In [4]:
# generate_email(api_key=openai_api,prompt='Genearte a email for friednly match to ronaldo',tone='Friendly')

In [5]:
def load_hf_pipelines():
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    generator = pipeline("text2text-generation", model="google/flan-t5-base")
    sentiment = pipeline("sentiment-analysis")
    ner = pipeline("ner", grouped_entities=True)
    return summarizer, generator, sentiment, ner

summarizer_hf, generator_hf, sentiment_hf, ner_hf = load_hf_pipelines()

Device set to use mps:0
Device set to use mps:0
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architect

In [6]:
def generate_email_with_hf(prompt):
    try:
        thread_summary = summarizer_hf(prompt, max_length=80, min_length=30, do_sample=False)[0]["summary_text"]
        reply_output = generator_hf(f"Reply to the email: {prompt}", max_length=256, do_sample=True)[0]["generated_text"]
        subject_line = generator_hf(f"Generate a short subject line for: {reply_output}", max_length=20)[0]["generated_text"]
        return {"error": None, "response": reply_output, "subject": subject_line, "summary": thread_summary}
    except Exception as e:
        return {"error": str(e), "response": None}

In [8]:
def extract_entities(text):
    entities = ner_hf(text)
    df = pd.DataFrame([{"entity": ent.get("entity_group", ent.get("entity")), "text": ent["word"], "score": ent.get("score", None)} for ent in entities])
    return df